# MeanHamilMinimizer_rigetti_autograd

* Feedback loop between Qubiter and Rigetti QVM
* minimization via autograd

>This notebook calls Rigetti's method QVMConnection() which only works if you first:
* install the Rigetti Forest SDK available at https://www.rigetti.com/forest
* open a second terminal (besides the one that runs this notebook) and type "qvm -S" in it
* open a third terminal and type "quilc -S" in it

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/rrtucci/PycharmProjects/qubiter/qubiter/jupyter_notebooks
/home/rrtucci/PycharmProjects/qubiter


In [2]:
import qubiter.adv_applications.setup_autograd # do this first!
from qubiter.adv_applications.MeanHamil_native import *
from qubiter.adv_applications.MeanHamil_rigetti import *
from qubiter.adv_applications.MeanHamilMinimizer import *

np installed? False
numpy installed? True
autograd.numpy installed? True
loaded OneQubitGate, WITH autograd.numpy
pu2 in dir True
pu2 in sys.modules False


In [3]:
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from pyquil import get_qc

In [4]:
qvm_url = "http://localhost:5000"
compiler_server_address = "tcp://localhost:5555"
forest_url = "https://forest-server.qcs.rigetti.com"
qvm = QVMConnection(endpoint=qvm_url, compiler_endpoint=compiler_server_address)

## First Example (taken from Pennylane docs). 

In [5]:
qc = get_qc('2q-qvm')

In [6]:
num_qbits = 2
file_prefix = 'mean_hamil_rigetti_test1'
emb = CktEmbedder(num_qbits, num_qbits)
wr = SEO_writer(file_prefix, emb)
wr.write_Rx(0, rads='#1')
wr.write_Ry(0, rads='#2')
wr.close_files()

In [7]:
wr.print_eng_file(jup=True)

In [8]:
wr.print_pic_file(jup=True)

In [9]:
fun_name_to_fun = None

In [10]:
hamil = QubitOperator('Z0', 1.)
print('hamil=\n', hamil)

hamil=
 1.0 [Z0]


In [11]:
init_var_num_to_rads = {1: .3, 2: .8}
all_var_nums = [1, 2]

In [12]:
num_samples = 100
print_hiatus = 1
verbose = False
np.random.seed(1234)

In [13]:
emp_mhamil = MeanHamil_rigetti(qc, file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, num_samples=num_samples)
targ_mhamil = MeanHamil_native(file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, simulator_name='SEO_simulator') # zero samples

In [14]:
mini = MeanHamilMinimizer(emp_mhamil, targ_mhamil,
                 all_var_nums, init_var_num_to_rads,
                 print_hiatus=print_hiatus, verbose=verbose)

In [15]:
emp_mhamil.translator.print_aqasm_file()


pg += RX(rads1*(-2), 0)
pg += RY(rads2*(-2), 0)




In [16]:
print(emp_mhamil.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[2]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RX(rads1*-2) 0
RY(rads2*-2) 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [17]:
mini.find_min(minlib='autograd', num_iter=10, descent_rate=.1)

x_val~ (#1, #2)
iter=0, cost=-0.060000, targ_cost=-0.024099, x_val=0.300000, 0.800000
iter=1, cost=-0.340000, targ_cost=-0.292577, x_val=0.296703, 0.965738
iter=2, cost=-0.620000, targ_cost=-0.551784, x_val=0.257234, 1.128646
iter=3, cost=-0.680000, targ_cost=-0.764436, x_val=0.194856, 1.271741
iter=4, cost=-0.880000, targ_cost=-0.896213, x_val=0.132062, 1.380452
iter=5, cost=-0.960000, targ_cost=-0.959154, x_val=0.083588, 1.453728
iter=6, cost=-1.000000, targ_cost=-0.984771, x_val=0.051216, 1.499885
iter=7, cost=-1.000000, targ_cost=-0.994446, x_val=0.030971, 1.528101
iter=8, cost=-1.000000, targ_cost=-0.997991, x_val=0.018635, 1.545146
iter=9, cost=-1.000000, targ_cost=-0.999275, x_val=0.011193, 1.555399


## Second, more complicated example

In [18]:
qc = get_qc('4q-qvm')

In [19]:
num_qbits = 4
file_prefix = 'mean_hamil_rigetti_test2'
emb = CktEmbedder(num_qbits, num_qbits)
wr = SEO_writer(file_prefix, emb)
wr.write_Ry(2, rads=np.pi/7)
wr.write_Rx(1, rads='#2*.5')
wr.write_Ry(1, rads='-#1*.3')
wr.write_cnot(2, 3)
wr.write_qbit_swap(1, 2)
wr.close_files()

In [20]:
wr.print_eng_file(jup=True)

In [21]:
wr.print_pic_file(jup=True)

In [22]:
fun_name_to_fun = None

In [23]:
hamil = QubitOperator('X1 Y3 X1 Y1 X2', .4) + QubitOperator('Y2 X1', .7)
print('hamil=\n', hamil)

hamil=
 0.7 [X1 Y2] +
0.4 [Y1 X2 Y3]


In [24]:
init_var_num_to_rads = {1: 2.1, 2: 3.4}
all_var_nums = [1, 2]

In [25]:
num_samples = 100
print_hiatus = 1
verbose = False
np.random.seed(1234)

In [26]:
emp_mhamil = MeanHamil_rigetti(qc, file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, num_samples=num_samples)
targ_mhamil = MeanHamil_native(file_prefix, num_qbits, hamil,
            all_var_nums, fun_name_to_fun, simulator_name='SEO_simulator') # zero samples

In [27]:
mini = MeanHamilMinimizer(emp_mhamil, targ_mhamil,
                 all_var_nums, init_var_num_to_rads,
                 print_hiatus=print_hiatus, verbose=verbose)

In [28]:
emp_mhamil.translator.print_aqasm_file()


pg += RY(-0.8975979109989651, 2)
pg += RX(rads2*.5*(-2), 1)
pg += RY(-rads1*.3*(-2), 1)
pg += CNOT(2, 3)
pg += SWAP(2, 1)




In [29]:
print(emp_mhamil.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[4]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RY(-0.8975979109989651) 2
RX(rads2*0.5*-2) 1
RY(-1*rads1*0.3*-2) 1
CNOT 2 3
SWAP 2 1
RY(-pi/2) 1
RX(pi/2) 2
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]
MEASURE 3 ro[3]



In [30]:
mini.find_min(minlib='autograd', num_iter=10, descent_rate=.1)

x_val~ (#1, #2)
iter=0, cost=-0.336000, targ_cost=-0.287864, x_val=2.100000, 3.400000
iter=1, cost=-0.316000, targ_cost=-0.288742, x_val=2.105548, 3.392383
iter=2, cost=-0.370000, targ_cost=-0.289583, x_val=2.111049, 3.384978
iter=3, cost=-0.192000, targ_cost=-0.290390, x_val=2.116503, 3.377780
iter=4, cost=-0.246000, targ_cost=-0.291163, x_val=2.121910, 3.370784
iter=5, cost=-0.310000, targ_cost=-0.291904, x_val=2.127269, 3.363985
iter=6, cost=-0.224000, targ_cost=-0.292615, x_val=2.132580, 3.357378
iter=7, cost=-0.426000, targ_cost=-0.293297, x_val=2.137843, 3.350958
iter=8, cost=-0.170000, targ_cost=-0.293951, x_val=2.143058, 3.344720
iter=9, cost=-0.274000, targ_cost=-0.294578, x_val=2.148225, 3.338660
